In [1]:
from planckmc.track_generation import halo_model
from planckmc.track_generation import make_tracks
from planckmc.detector_characteristics import DETECTOR_CHARACTERISTICS
from planckmc.response import sensor_response, RESPONSE_DICT
from planckmc.config import CONFIG
import planckanalysis.integral_transform as pint
import planckanalysis.integral_transform_reworked as pint2


import numpy as np
from numba import njit, jit
from tqdm import tqdm
import numericalunits as nu
from scipy.signal import peak_widths
from scipy.signal import peak_prominences
from random import randrange, uniform
import json

%matplotlib inline
import matplotlib # plotting libraries
from matplotlib import animation, rc, cm
import matplotlib.pyplot as plt
import matplotlib.colors as clr
from mpl_toolkits.mplot3d import Axes3D

import scipy
from scipy import signal

import seaborn as sn

#%pdb

C:\Users\bahaa\anaconda3\lib\site-packages\wimprates\halo.py:144: RuntimeWarning: divide by zero encountered in true_divide
  / (2 * v_earth_t * v))


In [6]:
scl_fct = 1e4 # Scale Factor for Optimized Time variables (entry time and time difference)
hnds_un = 1e-2 # Units to Hundreds -> Because Some values are already multiplied by one hundred (i.e. every unit is one hundred of the value)
tnsMS_ms = 1e-1 # Tens of microseconds to microseconds
s_ns = 1e9 # Seconds to nanoseconds
s_ms = 1e6 # Seconds to microseconds

In [7]:
#Essential Cell: SIMULATION ACCELERATION
#Creates track and sensor accelerations (without noise) & developes varibale name standards

#Custom Track: 7.5 (units) in length
#vel, entry_vecs, exit_vecs, t_entry, t_exit = np.array([257500.]), np.array([[2.10893786], [3.19662096], [3.51766905]]), np.array([[-3.94093443], [ 3.26567685], [-0.91890723]]), np.array([0.]), np.array([2.91362915e-05])

#Randomized Track:
vel = halo_model.generate_vel_array(n_vels=1) # The number infront of n_vels is how many tracks are produced, Keep at one for analysis
entry_vecs, exit_vecs, t_entry, t_exit = make_tracks.generate_tracks(vel, np.zeros(vel.shape))

#t_entry, t_exit are in seconds, the space variables are in meters

track=0 # Identifies that we are looking at the first track
radius = float(CONFIG['Track Generation']['BoundingSphereRadius'])

sensors = tuple(DETECTOR_CHARACTERISTICS.keys()) # Loads the sensor information

# Make sure the linear response in RESPONSE_DICT (simulation) is the same as the one used in the analysis (Run Response_update)
# Eventually, the linear response and the FIR filter used for the template generation, which is currenly this same one, will be the same
# as that of the sensors used in the physical experiment. By matching the two linear responses and FIR filters together, the matching can work

lin_resp = RESPONSE_DICT[sensors[0]]['linear_response']
lin_resp_len = len(lin_resp)

# The floor value is important because of two reasons: 1) it provides a way to analyze for entry time, 2) it makes the length of the data longer than
# the response length (which is CRUCIAL)
floor_val = ((lin_resp_len - (((t_exit - t_entry) * hnds_un * s_ns) + 2)) // 2) + 1

# Padding is in 10s of microseconds (nanoseconds/100)
if floor_val > 1:
    n_pad_strt = randrange(floor_val, floor_val + 50)
    n_pad_end = randrange(floor_val, floor_val + 50)
else:
    floor_val = 1
    n_pad_strt = randrange(floor_val, floor_val + 50)
    n_pad_end = randrange(floor_val, floor_val + 50)

# Acceleration without noise:
out = make_tracks.generate_acceleration_dict(entry_vecs, exit_vecs, t_entry, t_exit, {'M':1e8, 'G':6.67e-11}, strt_padding=n_pad_strt, end_padding=n_pad_end)

accels = []
sensors_pos = []
cnt = 0

# Below we initialize for the acceleration array and the sensor positions array
for key in out[track]:
    if key not in ['time', 'particle_location']:
        accels.append(out[track][sensors[cnt]])
        sensors_pos.append(DETECTOR_CHARACTERISTICS[key]['position'])
        cnt += 1

track_len = np.linalg.norm(exit_vecs - entry_vecs)

# Check: Helpful for when n_pad_strt & n_pad_end vary to check for sizes of arrays

# This conversion (of t_entry and of n_pad_strt) is what matches the padding and the time to be in nanoseconds -> (t_entry*s_ns)+(n_pad_strt/hnds_un)
sensor_response(sensors[0], out[0][sensors[0]]), track_len, n_pad_strt, n_pad_end, len(out[track]['time']), (t_entry*s_ns)+(n_pad_strt/hnds_un)

(array([[32767, 32767, 32768],
        [32767, 32767, 32768],
        [32767, 32767, 32769],
        ...,
        [32768, 32768, 32768],
        [32768, 32768, 32768],
        [32767, 32768, 32768]], dtype=int64),
 4.059231311160934,
 403,
 434,
 906,
 array([40300.]))

In [8]:
# Truth values for the entry and exit spacial cartesian parameters and entry and exit times
entry_vecs, exit_vecs, t_entry, t_exit

(array([[ 4.72295395],
        [-1.32924402],
        [ 1.72244485]]),
 array([[ 3.28706661],
        [-3.87877745],
        [-1.09099888]]),
 array([0.]),
 array([6.73731338e-06]))

In [10]:
# Essential cell: CREATES NOISY SIMULATION ACCELERATIONS
# Generates associated adc values (with added noise) to accelerations
# and takes adc values and returns associated accelerations with noise

accels_noise = []
data_info = sensor_response(sensors[0], out[track][sensors[0]])
full_data_1 = np.array([[0] * 3 for i in range(data_info.shape[0])])
full_data = np.array([full_data_1 for j in range(len(sensors))])
tm_vals = []

for sensrs in range(0,len(sensors)):
    data_info = sensor_response(sensors[sensrs], out[track][sensors[sensrs]], response_dict=RESPONSE_DICT)
    for tms in range(0,data_info.shape[0]):
        for dim in range(0,len(data_info[tms])):
            # adc with noise
            full_data[sensrs][tms][dim] = data_info[tms][dim]

# This is the transfer response which provides the look up dictionary for the procedure below
Transf_Resp = np.array(RESPONSE_DICT[sensors[0]]['signal_transfer_response'])

# Acceleration with noise determination; we concatenate the transfer response with the end values of -1 and 1 which correspond to the 'inf' bounds
accels_w_noise = pint.adc_readout_to_accel(full_data, np.concatenate([[-1], Transf_Resp[:-1]+np.diff(Transf_Resp)/2, [1]]))

for sensrs2 in range(0,len(sensors)):
    accels_noise.append(accels_w_noise[sensrs2])
    
time = out[track]['time']-out[track]['time'][0] # This variable is in nanoseconds

In [12]:
# Here we generate the corresponding truth angle spatial information of the track from the cartesian truth data

# Theta goes from 0 to 180 deg, and phi goes from -180 to 180 deg

alpha_theta = np.array([0, 0, 1])
cos_theta_entry = pint.py_ang(entry_vecs.T[0], alpha_theta.T)
if cos_theta_entry < -1:
    cos_theta_entry = -1

if cos_theta_entry > 1:
    cos_theta_entry = 1
theta_entry = np.arccos(cos_theta_entry) # This is the theta entry value

cos_theta_exit = pint.py_ang(exit_vecs.T[0], alpha_theta.T)
if cos_theta_exit < -1:
    cos_theta_exit = -1

if cos_theta_exit > 1:
    cos_theta_exit = 1
theta_exit = np.arccos(cos_theta_exit) # This is the theta exit value


alpha_phi = np.array([1, 0, 0])

# Below are the truth phi entry and exit values
phi_entry = np.arctan2(entry_vecs[1], entry_vecs[0])[0]
phi_exit = np.arctan2(exit_vecs[1], exit_vecs[0])[0]


t_0 = t_entry + ((n_pad_strt - 1) / (hnds_un * s_ns)) # This variable is in Seconds
d_t = t_exit - t_entry # This variable is in Seconds

In [19]:
# Check: Transforming the angle values back to cartesian

x_0 = radius * np.sin(theta_entry) * np.cos(phi_entry)
y_0 = radius * np.sin(theta_entry) * np.sin(phi_entry)
z_0 = radius * np.cos(theta_entry)

x_1 = radius * np.sin(theta_exit) * np.cos(phi_exit)
y_1 = radius * np.sin(theta_exit) * np.sin(phi_exit)
z_1 = radius * np.cos(theta_exit)

print(x_0, y_0, z_0, entry_vecs)
print(x_1, y_1, z_1, exit_vecs)

4.722953954169598 -1.3292440243631614 1.7224448526697702 [[ 4.72295395]
 [-1.32924402]
 [ 1.72244485]]
3.2870666090525793 -3.878777454789021 -1.0909988835337918 [[ 3.28706661]
 [-3.87877745]
 [-1.09099888]]


------------------------------------------------------------------

------------------------------------------------------------------

------------------------------------------------------------------

Wrapper 1 is only for entry time, all other variables are kept to their truth values; wrapper1_no_Intrp is without interpolation for time

In [30]:
def wrapper1_no_Intrp(alpha_def, theta_entry, phi_entry, theta_exit, phi_exit, accels=accels, lin_resp=lin_resp, sensors_pos=sensors_pos):
    S = []
    S_norm = []
        
    timestep_indices = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct))))])
    t_0_op = time[timestep_indices]

    
    theta_entry_op, phi_entry_op, theta_exit_op, phi_exit_op, d_t_op = theta_entry, phi_entry, theta_exit, phi_exit, d_t
    
    x_0 = radius * np.sin(theta_entry_op) * np.cos(phi_entry_op)
    y_0 = radius * np.sin(theta_entry_op) * np.sin(phi_entry_op)
    z_0 = radius * np.cos(theta_entry_op)
        
    x_1 = radius * np.sin(theta_exit_op) * np.cos(phi_exit_op)
    y_1 = radius * np.sin(theta_exit_op) * np.sin(phi_exit_op)
    z_1 = radius * np.cos(theta_exit_op)


    t_0_eff = t_0_op - t_0_op
    t_1_eff = np.float(d_t_op)
    
    alpha_pair = np.array([[float(x_0),
                           float(y_0),
                           float(z_0),
                           float(t_0_eff),
                           float(x_1),
                           float(y_1),
                           float(z_1),
                           float(t_1_eff)]])
    
    timesteps=t_0_op
                
    transformed_data = pint.transform(time/s_ns, accels_noise, timesteps/s_ns, timestep_indices, alpha_pair, sensors_pos, lin_resp)
    
    return -transformed_data['S_norm']

In [17]:
def wrapper1(alpha_def, theta_entry, phi_entry, theta_exit, phi_exit, accels=accels, lin_resp=lin_resp, sensors_pos=sensors_pos):
    S = []
    S_norm = []
    intrp_val = (alpha_def[0] * scl_fct) - time[np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct))))])]


    if intrp_val == 0:
        timestep_indices = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct))))])
        t_0_op = time[timestep_indices]


        theta_entry_op, phi_entry_op, theta_exit_op, phi_exit_op, d_t_op = theta_entry, phi_entry, theta_exit, phi_exit, d_t

        x_0 = radius * np.sin(theta_entry_op) * np.cos(phi_entry_op)
        y_0 = radius * np.sin(theta_entry_op) * np.sin(phi_entry_op)
        z_0 = radius * np.cos(theta_entry_op)

        x_1 = radius * np.sin(theta_exit_op) * np.cos(phi_exit_op)
        y_1 = radius * np.sin(theta_exit_op) * np.sin(phi_exit_op)
        z_1 = radius * np.cos(theta_exit_op)


        t_0_eff = t_0_op - t_0_op
        t_1_eff = np.float(d_t_op)

        alpha_pair = np.array([[float(x_0),
                               float(y_0),
                               float(z_0),
                               float(t_0_eff),
                               float(x_1),
                               float(y_1),
                               float(z_1),
                               float(t_1_eff)]])

        timesteps=t_0_op

        transformed_data = pint.transform(time/s_ns, accels_noise, timesteps/s_ns, timestep_indices, alpha_pair, sensors_pos, lin_resp)

        S = transformed_data['S_norm']
    else:
        if intrp_val > 0:
            timestep_indices1 = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct))))])
            t_0_op1 = time[timestep_indices1]
            
            timestep_indices2 = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct)))) + 1])
            
            print(timestep_indices2)
            
            if timestep_indices2 < len(time):
                t_0_op2 = time[timestep_indices2]
            else:
                timestep_indices2 = np.array([len(time) - 1])
                t_0_op2 = np.array([np.max(time)])

            blnc_val = intrp_val/(t_0_op2-t_0_op1)
        elif intrp_val < 0:
            timestep_indices1 = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct))))])
            t_0_op1 = time[timestep_indices1]

            timestep_indices2 = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct)))) - 1])
            
            if timestep_indices2 >= 0:
                t_0_op2 = time[timestep_indices2]
            else:
                timestep_indices2 = np.array([0])
                t_0_op2 = np.array([0])

            blnc_val = intrp_val/(t_0_op1 - t_0_op2)

        theta_entry_op, phi_entry_op, theta_exit_op, phi_exit_op, d_t_op = theta_entry, phi_entry, theta_exit, phi_exit, d_t

        x_0 = radius * np.sin(theta_entry_op) * np.cos(phi_entry_op)
        y_0 = radius * np.sin(theta_entry_op) * np.sin(phi_entry_op)
        z_0 = radius * np.cos(theta_entry_op)

        x_1 = radius * np.sin(theta_exit_op) * np.cos(phi_exit_op)
        y_1 = radius * np.sin(theta_exit_op) * np.sin(phi_exit_op)
        z_1 = radius * np.cos(theta_exit_op)


        t_0_eff = 0 #t_0_op - t_0_op
        t_1_eff = np.float(d_t_op)

        alpha_pair = np.array([[float(x_0),
                               float(y_0),
                               float(z_0),
                               float(t_0_eff),
                               float(x_1),
                               float(y_1),
                               float(z_1),
                               float(t_1_eff)]])

        timesteps1=t_0_op1
        transformed_data1 = pint.transform(time/s_ns, accels_noise, timesteps1/s_ns, timestep_indices1, alpha_pair, sensors_pos, lin_resp)

        timesteps2=t_0_op2
        transformed_data2 = pint.transform(time/s_ns, accels_noise, timesteps2/s_ns, timestep_indices2, alpha_pair, sensors_pos, lin_resp)


        S1 = (1 - blnc_val) * transformed_data1['S_norm']
        S2 = blnc_val * transformed_data2['S_norm']
        S = S1 + S2
    return -S

------------------------------------------------------------------

Wrapper 2 is for velocity/time difference, all other variables are kept to their truth values

In [31]:
def wrapper2(alpha_def, theta_entry, phi_entry, theta_exit, phi_exit, accels=accels, lin_resp=lin_resp, sensors_pos=sensors_pos):
    S = []
    S_norm = []
        
    timestep_indices = np.array([np.int(np.argmin(np.abs(time-(t_0 * s_ns))))])
    t_0_op = time[timestep_indices] / s_ns
    
    d_t_op = alpha_def[0] * scl_fct
    
    theta_entry_op, phi_entry_op, theta_exit_op, phi_exit_op = theta_entry, phi_entry, theta_exit, phi_exit
    
    x_0 = radius * np.sin(theta_entry_op) * np.cos(phi_entry_op)
    y_0 = radius * np.sin(theta_entry_op) * np.sin(phi_entry_op)
    z_0 = radius * np.cos(theta_entry_op)
        
    x_1 = radius * np.sin(theta_exit_op) * np.cos(phi_exit_op)
    y_1 = radius * np.sin(theta_exit_op) * np.sin(phi_exit_op)
    z_1 = radius * np.cos(theta_exit_op)


    t_0_eff = t_0_op - t_0_op
    t_1_eff = np.float(d_t_op) / s_ns
    
    alpha_pair = np.array([[float(x_0),
                           float(y_0),
                           float(z_0),
                           float(t_0_eff),
                           float(x_1),
                           float(y_1),
                           float(z_1),
                           float(t_1_eff)]])
    
    timesteps=t_0_op
                
    transformed_data = pint.transform(time/s_ns, accels_noise, timesteps, timestep_indices, alpha_pair, sensors_pos, lin_resp)
    
    # NOTE: The time components of transformed data are coming out in seconds
    
    return -transformed_data['S_norm']

------------------------------------------------------------------

Wrapper 3 is for spatial analysis, all other variables are kept to their truth values

In [32]:
def wrapper3(alpha_def, accels=accels, lin_resp=lin_resp, sensors_pos=sensors_pos):
    S = []
    S_norm = []
        
    timestep_indices = np.array([np.int(np.argmin(np.abs(time-(t_0 * s_ns))))])
    t_0_op = time[timestep_indices] / s_ns
        
    theta_entry_op, phi_entry_op, theta_exit_op, phi_exit_op, d_t_op = alpha_def[0], alpha_def[1], alpha_def[2], alpha_def[3], d_t * s_ns
    
    
    x_0 = radius * np.sin(theta_entry_op) * np.cos(phi_entry_op)
    y_0 = radius * np.sin(theta_entry_op) * np.sin(phi_entry_op)
    z_0 = radius * np.cos(theta_entry_op)
        
    x_1 = radius * np.sin(theta_exit_op) * np.cos(phi_exit_op)
    y_1 = radius * np.sin(theta_exit_op) * np.sin(phi_exit_op)
    z_1 = radius * np.cos(theta_exit_op)


    t_0_eff = t_0_op - t_0_op
    t_1_eff = np.float(d_t_op) / s_ns
    
    alpha_pair = np.array([[float(x_0),
                           float(y_0),
                           float(z_0),
                           float(t_0_eff),
                           float(x_1),
                           float(y_1),
                           float(z_1),
                           float(t_1_eff)]])
    
    timesteps=t_0_op
            
    transformed_data = pint.transform(time/s_ns, accels_noise, timesteps, timestep_indices, alpha_pair, sensors_pos, lin_resp)
        
    return -transformed_data['S_norm']

------------------------------------------------------------------

Wrapper 2P is for all time related analysis (not spatial, which are kept to their truth values), 2P_no_Intrp is without time interpolation

In [33]:
# Without interpolation
def wrapper_2P_no_Intrp(alpha_def, theta_entry, phi_entry, theta_exit, phi_exit, accels=accels, lin_resp=lin_resp, sensors_pos=sensors_pos):
    S = []
    S_norm = []
        
    timestep_indices = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct))))])
    t_0_op = time[timestep_indices] / s_ns
    
    d_t_op = alpha_def[1] * scl_fct
    
    theta_entry_op, phi_entry_op, theta_exit_op, phi_exit_op = theta_entry, phi_entry, theta_exit, phi_exit
    
    x_0 = radius * np.sin(theta_entry_op) * np.cos(phi_entry_op)
    y_0 = radius * np.sin(theta_entry_op) * np.sin(phi_entry_op)
    z_0 = radius * np.cos(theta_entry_op)
        
    x_1 = radius * np.sin(theta_exit_op) * np.cos(phi_exit_op)
    y_1 = radius * np.sin(theta_exit_op) * np.sin(phi_exit_op)
    z_1 = radius * np.cos(theta_exit_op)


    t_0_eff = t_0_op - t_0_op
    t_1_eff = np.float(d_t_op) / s_ns
    
    alpha_pair = np.array([[float(x_0),
                           float(y_0),
                           float(z_0),
                           float(t_0_eff),
                           float(x_1),
                           float(y_1),
                           float(z_1),
                           float(t_1_eff)]])
    
    timesteps=t_0_op
                
    transformed_data = pint.transform(time/s_ns, accels_noise, timesteps, timestep_indices, alpha_pair, sensors_pos, lin_resp)
    print(alpha_def[0], alpha_def[1])

    return -transformed_data['S_norm']

In [34]:
def wrapper_2P(alpha_def, theta_entry, phi_entry, theta_exit, phi_exit, accels=accels, lin_resp=lin_resp, sensors_pos=sensors_pos):
    S = []
    S_norm = []
    intrp_val = (alpha_def[0] * scl_fct) - time[np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct))))])]


    if intrp_val == 0:
        timestep_indices = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct))))])
        t_0_op = time[timestep_indices] / s_ns

        d_t_op = alpha_def[1] * scl_fct

        theta_entry_op, phi_entry_op, theta_exit_op, phi_exit_op = theta_entry, phi_entry, theta_exit, phi_exit

        x_0 = radius * np.sin(theta_entry_op) * np.cos(phi_entry_op)
        y_0 = radius * np.sin(theta_entry_op) * np.sin(phi_entry_op)
        z_0 = radius * np.cos(theta_entry_op)

        x_1 = radius * np.sin(theta_exit_op) * np.cos(phi_exit_op)
        y_1 = radius * np.sin(theta_exit_op) * np.sin(phi_exit_op)
        z_1 = radius * np.cos(theta_exit_op)


        t_0_eff = t_0_op - t_0_op
        t_1_eff = np.float(d_t_op) / s_ns

        alpha_pair = np.array([[float(x_0),
                               float(y_0),
                               float(z_0),
                               float(t_0_eff),
                               float(x_1),
                               float(y_1),
                               float(z_1),
                               float(t_1_eff)]])

        timesteps=t_0_op

        transformed_data = pint.transform(time/s_ns, accels_noise, timesteps, timestep_indices, alpha_pair, sensors_pos, lin_resp)

        S = transformed_data['S_norm']
    else:
        if intrp_val > 0:
            timestep_indices1 = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct))))])
            t_0_op1 = time[timestep_indices1] / s_ns
            
            timestep_indices2 = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct)))) + 1])
                        
            if timestep_indices2 < len(time):
                t_0_op2 = time[timestep_indices2] / s_ns
            else:
                timestep_indices2 = np.array([len(time) - 1])
                t_0_op2 = np.array([np.max(time)]) / s_ns

            blnc_val = intrp_val/(t_0_op2-t_0_op1)
        elif intrp_val < 0:
            timestep_indices1 = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct))))])
            t_0_op1 = time[timestep_indices1] / s_ns

            timestep_indices2 = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct)))) - 1])
            
            if timestep_indices2 >= 0:
                t_0_op2 = time[timestep_indices2] / s_ns
            else:
                timestep_indices2 = np.array([0])
                t_0_op2 = np.array([0]) / s_ns

            blnc_val = intrp_val/(t_0_op1 - t_0_op2)

        d_t_op = alpha_def[1] * scl_fct

        theta_entry_op, phi_entry_op, theta_exit_op, phi_exit_op = theta_entry, phi_entry, theta_exit, phi_exit

        x_0 = radius * np.sin(theta_entry_op) * np.cos(phi_entry_op)
        y_0 = radius * np.sin(theta_entry_op) * np.sin(phi_entry_op)
        z_0 = radius * np.cos(theta_entry_op)

        x_1 = radius * np.sin(theta_exit_op) * np.cos(phi_exit_op)
        y_1 = radius * np.sin(theta_exit_op) * np.sin(phi_exit_op)
        z_1 = radius * np.cos(theta_exit_op)


        t_0_eff = 0 #t_0_op - t_0_op
        t_1_eff = np.float(d_t_op) / s_ns

        alpha_pair = np.array([[float(x_0),
                               float(y_0),
                               float(z_0),
                               float(t_0_eff),
                               float(x_1),
                               float(y_1),
                               float(z_1),
                               float(t_1_eff)]])

        timesteps1=t_0_op1
        transformed_data1 = pint.transform(time/s_ns, accels_noise, timesteps1, timestep_indices1, alpha_pair, sensors_pos, lin_resp)

        timesteps2=t_0_op2
        transformed_data2 = pint.transform(time/s_ns, accels_noise, timesteps2, timestep_indices2, alpha_pair, sensors_pos, lin_resp)


        S1 = (1 - blnc_val) * transformed_data1['S_norm']
        S2 = blnc_val * transformed_data2['S_norm']
        S = S1 + S2
        
        print(alpha_def[0], alpha_def[1])
    return -S

------------------------------------------------------------------

Wrapper 6P is for all 6 parameters, 6P_no_Intrp is without time interpolation

In [35]:
def wrapper_6P_no_Intrp(alpha_def, accels=accels, lin_resp=lin_resp, sensors_pos=sensors_pos):
    S = []
    S_norm = []
        
    timestep_indices = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct))))])
    t_0_op = time[timestep_indices] / s_ns
    
    d_t_op = alpha_def[1] * scl_fct
    
    theta_entry_op, phi_entry_op, theta_exit_op, phi_exit_op = alpha_def[2], alpha_def[3], alpha_def[4], alpha_def[5]
    
    x_0 = radius * np.sin(theta_entry_op) * np.cos(phi_entry_op)
    y_0 = radius * np.sin(theta_entry_op) * np.sin(phi_entry_op)
    z_0 = radius * np.cos(theta_entry_op)
        
    x_1 = radius * np.sin(theta_exit_op) * np.cos(phi_exit_op)
    y_1 = radius * np.sin(theta_exit_op) * np.sin(phi_exit_op)
    z_1 = radius * np.cos(theta_exit_op)


    t_0_eff = t_0_op - t_0_op
    t_1_eff = np.float(d_t_op) / s_ns
    
    alpha_pair = np.array([[float(x_0),
                           float(y_0),
                           float(z_0),
                           float(t_0_eff),
                           float(x_1),
                           float(y_1),
                           float(z_1),
                           float(t_1_eff)]])
    
    timesteps=t_0_op
                
    transformed_data = pint.transform(time/s_ns, accels_noise, timesteps, timestep_indices, alpha_pair, sensors_pos, lin_resp)
        
    return -transformed_data['S_norm']

In [20]:
def wrapper_6P(alpha_def, accels=accels, lin_resp=lin_resp, sensors_pos=sensors_pos):
    S = []
    S_norm = []
    intrp_val = (alpha_def[0] * scl_fct) - time[np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct))))]


    if intrp_val == 0:
        timestep_indices = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct))))])
        t_0_op = time[timestep_indices] / s_ns

        d_t_op = alpha_def[1] * scl_fct

        theta_entry_op, phi_entry_op, theta_exit_op, phi_exit_op = alpha_def[2], alpha_def[3], alpha_def[4], alpha_def[5]

        x_0 = radius * np.sin(theta_entry_op) * np.cos(phi_entry_op)
        y_0 = radius * np.sin(theta_entry_op) * np.sin(phi_entry_op)
        z_0 = radius * np.cos(theta_entry_op)

        x_1 = radius * np.sin(theta_exit_op) * np.cos(phi_exit_op)
        y_1 = radius * np.sin(theta_exit_op) * np.sin(phi_exit_op)
        z_1 = radius * np.cos(theta_exit_op)


        t_0_eff = t_0_op - t_0_op
        t_1_eff = np.float(d_t_op) / s_ns

        alpha_pair = np.array([[float(x_0),
                               float(y_0),
                               float(z_0),
                               float(t_0_eff),
                               float(x_1),
                               float(y_1),
                               float(z_1),
                               float(t_1_eff)]])

        timesteps=t_0_op

        transformed_data = pint.transform(time/s_ns, accels_noise, timesteps, timestep_indices, alpha_pair, sensors_pos, lin_resp)

        S = transformed_data['S_norm']
    else:
        if intrp_val > 0:
            timestep_indices1 = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct))))])
            t_0_op1 = time[timestep_indices1] / s_ns
            
            timestep_indices2 = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct)))) + 1])
                        
            if timestep_indices2 < len(time):
                t_0_op2 = time[timestep_indices2] / s_ns
            else:
                timestep_indices2 = np.array([len(time) - 1])
                t_0_op2 = np.array([np.max(time)]) / s_ns

            blnc_val = intrp_val/(t_0_op2-t_0_op1)
        elif intrp_val < 0:
            timestep_indices1 = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct))))])
            t_0_op1 = time[timestep_indices1] / s_ns

            timestep_indices2 = np.array([np.int(np.argmin(np.abs(time-(alpha_def[0] * scl_fct)))) - 1])
            
            if timestep_indices2 >= 0:
                t_0_op2 = time[timestep_indices2] / s_ns
            else:
                timestep_indices2 = np.array([0])
                t_0_op2 = np.array([0]) / s_ns

            blnc_val = intrp_val/(t_0_op1 - t_0_op2)

        d_t_op = alpha_def[1] * scl_fct

        theta_entry_op, phi_entry_op, theta_exit_op, phi_exit_op = alpha_def[2], alpha_def[3], alpha_def[4], alpha_def[5]

        x_0 = radius * np.sin(theta_entry_op) * np.cos(phi_entry_op)
        y_0 = radius * np.sin(theta_entry_op) * np.sin(phi_entry_op)
        z_0 = radius * np.cos(theta_entry_op)

        x_1 = radius * np.sin(theta_exit_op) * np.cos(phi_exit_op)
        y_1 = radius * np.sin(theta_exit_op) * np.sin(phi_exit_op)
        z_1 = radius * np.cos(theta_exit_op)


        t_0_eff = 0 #t_0_op - t_0_op
        t_1_eff = np.float(d_t_op) / s_ns

        alpha_pair = np.array([[float(x_0),
                               float(y_0),
                               float(z_0),
                               float(t_0_eff),
                               float(x_1),
                               float(y_1),
                               float(z_1),
                               float(t_1_eff)]])

        timesteps1=t_0_op1
        transformed_data1 = pint.transform(time/s_ns, accels_noise, timesteps1, timestep_indices1, alpha_pair, sensors_pos, lin_resp)

        timesteps2=t_0_op2
        transformed_data2 = pint.transform(time/s_ns, accels_noise, timesteps2, timestep_indices2, alpha_pair, sensors_pos, lin_resp)


        S1 = (1 - blnc_val) * transformed_data1['S_norm']
        S2 = blnc_val * transformed_data2['S_norm']
        S = S1 + S2
        
        print(alpha_def[0], alpha_def[1], alpha_def[2], alpha_def[3], alpha_def[4], alpha_def[5])

    return -S

------------------------------------------------------------------

------------------------------------------------------------------

------------------------------------------------------------------

In [37]:
theta_entry, theta_exit, (t_entry*s_ns)+(n_pad_strt/hnds_un), phi_entry, phi_exit, (t_exit-t_entry)*s_ns # Times are in nanoseconds

(0.4768822862810498,
 1.5463971802245688,
 array([32200.]),
 -0.8167292960052399,
 -3.135141681595092,
 array([20524.42996829]))

------------------------------------------------------------------

------------------------------------------------------------------

In all of the optimization analyses below, the truth values are identified first and then the particular optimizer is utilized, sometimes with bounds, and with initial estimates. The important results are the output varible values which are supposed to corresspond to their truth equivalents, and for some optimizers an inverse hessian matrix. The inverse hessian produces an accurate error analysis as a covariance matrix if the analyzed signal value in the wrappers above is S_norm, which is supposed to be a correct SNR value, normalizing the signal output by the noise in a log likelihood.

------------------------------------------------------------------

------------------------------------------------------------------

In [34]:
((t_entry*s_ns)+(n_pad_strt/hnds_un))/scl_fct

array([3.31])

As will be seen in some of the parameter analyses below, the Powell algorithm is used to approach the correct truth value first and then the result is passed to the L-BFGS-B algorithm. This was a suggested method to produce an inverse hessian out of the Powell analysis (which itself does not provide an inv. hess.) as the L-BFGS-B analysis on its own from an arbitrary guess does not succeed in approaching a solution. However, since not many iterations of differentiation occur in the second part of the procedure in the L-BFGS-B part, the inv. hess. seems to be the default one.

In [35]:
bnds = [(0, 10)]
result_wr1 = scipy.optimize.minimize(wrapper1, [0], args=(theta_entry, phi_entry, theta_exit, phi_exit, accels, lin_resp, sensors_pos), method='Powell', bounds=bnds, options={'gtol': 1e-20})

C:\Users\bahaa\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:522: RuntimeWarning: Method Powell cannot handle constraints nor bounds.
  RuntimeWarning)
C:\Users\bahaa\anaconda3\lib\site-packages\ipykernel_launcher.py:2: OptimizeWarning: Unknown solver options: gtol
  
  0%|          | 0/1 [00:00<?, ?it/s]

[389]
[593]


  0%|          | 0/1 [00:00<?, ?it/s]

[389]
[467]


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

[332]


  0%|          | 0/1 [00:00<?, ?it/s]

[332]


100%|██████████| 1/1 [00:00<00:00, 11.80it/s]


In [36]:
bnds = [(0, 10)]
result_wr1_f = scipy.optimize.minimize(wrapper1, [result_wr1.x], args=(theta_entry, phi_entry, theta_exit, phi_exit, accels, lin_resp, sensors_pos), method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-20})

100%|██████████| 1/1 [00:00<00:00, 11.80it/s]

[332]



  0%|          | 0/1 [00:00<?, ?it/s]

[332]


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

[332]
[332]


  0%|          | 0/1 [00:00<?, ?it/s]

[332]
[332]


100%|██████████| 1/1 [00:00<00:00, 10.23it/s]


In [37]:
bnds = [(0, 10)]
result_wr1_ex = scipy.optimize.minimize(wrapper1, [((t_entry*s_ns)+(n_pad_strt/hnds_un))/scl_fct], args=(theta_entry, phi_entry, theta_exit, phi_exit, accels, lin_resp, sensors_pos), method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-20})

  0%|          | 0/1 [00:00<?, ?it/s]

[332]


  0%|          | 0/1 [00:00<?, ?it/s]

[332]


100%|██████████| 1/1 [00:00<00:00, 10.08it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

[332]
[332]


  0%|          | 0/1 [00:00<?, ?it/s]

[332]
[332]


100%|██████████| 1/1 [00:00<00:00,  9.20it/s]


In [38]:
result_wr1.x, result_wr1_f.x, result_wr1_ex.x

array(3.3150008)

In [40]:
np.sqrt(np.diag(result_wr1_f.hess_inv.todense())) #err

array([1.])

In [42]:
np.sqrt(np.diag(result_wr1_ex.hess_inv.todense())) #err

array([1.])

------------------------------------------------------------------

------------------------------------------------------------------

In [43]:
((t_exit-t_entry)*s_ns)/scl_fct

array([2.54824504])

In [44]:
bnds = [(0, 10)]
result_wr2 = scipy.optimize.minimize(wrapper2, [0], args=(theta_entry, phi_entry, theta_exit, phi_exit, accels, lin_resp, sensors_pos), method='Powell', bounds=bnds, options={'gtol': 1e-20})

C:\Users\bahaa\anaconda3\lib\site-packages\ipykernel_launcher.py:2: OptimizeWarning: Unknown solver options: gtol
  
  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\bahaa\anaconda3\lib\site-packages\planckanalysis\integral_transform.py:205: RuntimeWarning: divide by zero encountered in true_divide
  dir_vector_step = dir_vector / (alpha_pair[7] - alpha_pair[3]) * adc_timestep_size
100%|██████████| 1/1 [00:00<00:00, 10.90it/s]


In [45]:
bnds = [(0, 10)]
result_wr2_f = scipy.optimize.minimize(wrapper2, [result_wr2.x], args=(theta_entry, phi_entry, theta_exit, phi_exit, accels, lin_resp, sensors_pos), method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-20})

100%|██████████| 1/1 [00:00<00:00, 10.44it/s]


In [46]:
bnds = [(0, 10)]
result_wr2_ex = scipy.optimize.minimize(wrapper2, [((t_exit-t_entry)*s_ns)/scl_fct], args=(theta_entry, phi_entry, theta_exit, phi_exit, accels, lin_resp, sensors_pos), method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-20})

100%|██████████| 1/1 [00:00<00:00, 13.19it/s]


In [47]:
result_wr2.x, result_wr2_ex.x

array(2.46999966)

In [48]:
np.sqrt(np.diag(result_wr2_f.hess_inv.todense())) #err

array([1.])

In [50]:
np.sqrt(np.diag(result_wr2_ex.hess_inv.todense())) #err

array([1.])

------------------------------------------------------------------

------------------------------------------------------------------

In [51]:
theta_entry, phi_entry, theta_exit, phi_exit

(0.9073887605731223,
 -2.7303964599031776,
 1.7721112213198442,
 0.328134705686381)

In [52]:
bnds = [(0, 3.14), (-3.14, 3.14), (0, 3.14), (-3.14, 3.14)]
result_wr3 = scipy.optimize.minimize(wrapper3, [0, 0, 0, 0], args=(accels, lin_resp, sensors_pos), method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-12})

100%|██████████| 1/1 [00:00<00:00, 11.66it/s]


In [53]:
bnds = [(-3.14, 3.14), (-3.14, 3.14), (-3.14, 3.14), (-3.14, 3.14)]
result_wr3_ex = scipy.optimize.minimize(wrapper3, [theta_entry, phi_entry, theta_exit, phi_exit], args=(accels, lin_resp, sensors_pos), method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-12})

100%|██████████| 1/1 [00:00<00:00, 13.02it/s]


In [54]:
result_wr3.x, result_wr3_ex.x

array([0., 0., 0., 0.])

In [55]:
np.sqrt(np.diag(result_wr3.hess_inv.todense())) #err

array([1., 1., 1., 1.])

In [57]:
result_wr3_ex.hess_inv.todense()

array([[ 2.36099533e-03, -1.19760235e-03, -2.77898664e-02,
        -1.31735644e-02],
       [-1.19760235e-03,  4.42500555e-02, -1.55830144e-02,
         3.32409838e-01],
       [-2.77898664e-02, -1.55830144e-02,  3.47281415e-01,
        -6.64539592e-02],
       [-1.31735644e-02,  3.32409838e-01, -6.64539591e-02,
         2.50458139e+00]])

In [58]:
np.sqrt(np.diag(result_wr3_ex.hess_inv.todense())) #err

array([0.04859007, 0.21035697, 0.58930588, 1.58258693])

------------------------------------------------------------------

------------------------------------------------------------------

In [59]:
((t_entry*s_ns)+(n_pad_strt/hnds_un))/scl_fct, ((t_exit-t_entry)*s_ns)/scl_fct

(array([3.31]), array([2.54824504]))

In [65]:
bnds = [(0,3), (0, 10)]
result_2P = scipy.optimize.minimize(wrapper_2P, [0, 0], args=(theta_entry, phi_entry, theta_exit, phi_exit, accels, lin_resp, sensors_pos), method='Powell', bounds=bnds, options={'gtol': 1e-20})

C:\Users\bahaa\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:522: RuntimeWarning: Method Powell cannot handle constraints nor bounds.
  RuntimeWarning)
C:\Users\bahaa\anaconda3\lib\site-packages\ipykernel_launcher.py:2: OptimizeWarning: Unknown solver options: gtol
  
100%|██████████| 1/1 [00:00<00:00, 14.75it/s]

2.6180339999999998 0.0
1.6180339748439998 0.0
1.9999999844527365 0.0
2.2360679652352626 0.0
2.38196600367021 0.0
2.4721359519563144 0.0
2.527864045775369 0.0
2.5623059025107344 0.0
2.587928961545842 0.0
2.587928961545842 0.0
2.587928961545842 1.0



  0%|          | 0/1 [00:00<?, ?it/s]

2.587928961545842 2.6180339999999998
2.587928961545842 2.265308835298585


100%|██████████| 1/1 [00:00<00:00, 15.92it/s]

...

2.6350000000100455 1.9600032427451906
2.6350000000100455 1.9599959292344655


  0%|          | 0/1 [00:00<?, ?it/s]

2.6350000000100455 1.960000449232569
2.6350000000100455 1.9599976557201315


100%|██████████| 1/1 [00:00<00:00, 17.59it/s]

2.6350000000100455 1.9599993822057271


In [58]:
bnds = [(0,3), (0, 10)]
result_2P_f = scipy.optimize.minimize(wrapper_2P, list(result_2P.x), args=(theta_entry, phi_entry, theta_exit, phi_exit, accels, lin_resp, sensors_pos), method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-20})

  0%|          | 0/1 [00:00<?, ?it/s]

2.6350000000100455 1.9599993822057271


  0%|          | 0/1 [00:00<?, ?it/s]

2.6350000100100455 1.9599993822057271


  0%|          | 0/1 [00:00<?, ?it/s]

...

2.635000010001067 1.959999382233123
2.635000000001067 1.959999392233123


  0%|          | 0/1 [00:00<?, ?it/s]

2.635000000000834 1.9599993822338344
2.6350000100008337 1.9599993822338344


100%|██████████| 1/1 [00:00<00:00, 14.31it/s]

2.635000000000834 1.9599993922338343


In [62]:
bnds = [(0,3), (0, 10)]
result_2P_ex = scipy.optimize.minimize(wrapper_2P, [((t_entry*s_ns)+(n_pad_strt/hnds_un))/scl_fct, ((t_exit-t_entry)*s_ns)/scl_fct], args=(theta_entry, phi_entry, theta_exit, phi_exit, accels, lin_resp, sensors_pos), method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-20})

  0%|          | 0/1 [00:00<?, ?it/s]

[301]


  0%|          | 0/1 [00:00<?, ?it/s]

[301]


  0%|          | 0/1 [00:00<?, ?it/s]

...

100%|██████████| 1/1 [00:00<00:00,  9.83it/s]

[301]


100%|██████████| 1/1 [00:00<00:00, 11.39it/s]

[301]



100%|██████████| 1/1 [00:00<00:00, 12.23it/s]


In [59]:
result_2P_f.x, result_2P_ex.x

array([2.635     , 1.95999938])

In [60]:
np.sqrt(np.diag(result_2P_f.hess_inv.todense())) #err

array([1., 1.])

In [66]:
np.sqrt(np.diag(result_2P_ex.hess_inv.todense())) #err

array([1., 1.])

------------------------------------------------------------------

------------------------------------------------------------------

In [67]:
((t_entry*s_ns)+(n_pad_strt/hnds_un))/scl_fct, ((t_exit-t_entry)*s_ns)/scl_fct, theta_entry, phi_entry, theta_exit, phi_exit

(array([3.31]),
 array([2.54824504]),
 0.9073887605731223,
 -2.7303964599031776,
 1.7721112213198442,
 0.328134705686381)

In [68]:
bnds = [(0,10), (0, 10), (0, 3.14), (-3.14, 3.14), (0, 3.14), (-3.14, 3.14)]
result_6P_ex_P = scipy.optimize.minimize(wrapper_6P, [((t_entry*s_ns)+(n_pad_strt/hnds_un))/scl_fct, ((t_exit-t_entry)*s_ns)/scl_fct, theta_entry, phi_entry, theta_exit, phi_exit], args=(accels, lin_resp, sensors_pos), method='SLSQP', bounds=bnds, options={'gtol': 1e-10})

C:\Users\bahaa\anaconda3\lib\site-packages\ipykernel_launcher.py:2: OptimizeWarning: Unknown solver options: gtol
  
  0%|          | 0/1 [00:00<?, ?it/s]

[301]


100%|██████████| 1/1 [00:00<00:00, 10.78it/s]


In [69]:
bnds = [(0,10), (0, 10), (0, 3.14), (-3.14, 3.14), (0, 3.14), (-3.14, 3.14)]
result_6P_ex_BF_fromP = scipy.optimize.minimize(wrapper_6P, list(result_6P_ex_P.x), args=(accels, lin_resp, sensors_pos), method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-20})

  0%|          | 0/1 [00:00<?, ?it/s]

[301]


  0%|          | 0/1 [00:00<?, ?it/s]

[301]


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

[301]


  0%|          | 0/1 [00:00<?, ?it/s]

[301]


100%|██████████| 1/1 [00:00<00:00, 10.55it/s]


In [70]:
result_6P_ex_BF_fromP.x

array([ 3.        ,  2.55596709,  0.86695406, -2.81036684,  1.76859947,
        0.34495404])

In [71]:
np.sqrt(np.diag(result_6P_ex_BF_fromP.hess_inv.todense())) #err

array([1.00000000e+00, 3.42308022e+05, 2.25174199e+06, 1.14808711e+06,
       4.80054729e+05, 3.32224986e+06])

In [72]:
bnds = [(0,10), (0, 10), (0, 3.14), (-3.14, 3.14), (0, 3.14), (-3.14, 3.14)]
result_6P_ex_BF = scipy.optimize.minimize(wrapper_6P, [((t_entry*s_ns)+(n_pad_strt/hnds_un))/scl_fct, ((t_exit-t_entry)*s_ns)/scl_fct, theta_entry, phi_entry, theta_exit, phi_exit], args=(accels, lin_resp, sensors_pos), method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-20})

  0%|          | 0/1 [00:00<?, ?it/s]

[3.e-05] [3.01e-05]
3.00000001 2.0524429968291673 0.4768822862810498 -0.8167292960052399 1.5463971802245688 -3.135141681595092


  0%|          | 0/1 [00:00<?, ?it/s]

[3.e-05] [3.01e-05]
3.00000001 2.0888672645977873 -0.49616737054060633 -0.696469054656649 1.0850057926284138 -3.14


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

[3.e-05] [3.01e-05]


KeyboardInterrupt: 

> c:\users\bahaa\anaconda3\lib\site-packages\numpy\core\numeric.py(817)convolve()
    815         raise ValueError('v cannot be empty')
    816     mode = _mode_from_name(mode)
--> 817     return multiarray.correlate(a, v[::-1], mode)
    818 
    819 



ipdb>  exit


In [73]:
result_6P_ex_BF.x

array([ 3.        ,  2.55596709,  0.86695406, -2.81036684,  1.76859947,
        0.34495404])

In [74]:
np.sqrt(np.diag(result_6P_ex_BF.hess_inv.todense())) #err

array([1.00000000e+00, 3.42308022e+05, 2.25174199e+06, 1.14808711e+06,
       4.80054729e+05, 3.32224986e+06])

In [75]:
bnds = [(0,10), (0, 10), (0, 3.14), (-3.14, 3.14), (0, 3.14), (-3.14, 3.14)]
result_6P_BF = scipy.optimize.minimize(wrapper_6P, [0, 0, 0, 0, 0, 0], args=(accels, lin_resp, sensors_pos), method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-20})

  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\bahaa\anaconda3\lib\site-packages\planckanalysis\integral_transform.py:205: RuntimeWarning: invalid value encountered in true_divide
  dir_vector_step = dir_vector / (alpha_pair[7] - alpha_pair[3]) * adc_timestep_size
100%|██████████| 1/1 [00:00<00:00, 1004.14it/s]

[1]


In [76]:
bnds = [(0,10), (0, 10), (0, 3.14), (-3.14, 3.14), (0, 3.14), (-3.14, 3.14)]
result_6P_P = scipy.optimize.minimize(wrapper_6P, [0, 0, 0, 0, 0, 0], args=(accels, lin_resp, sensors_pos), method='SLSQP', bounds=bnds, options={'gtol': 1e-10})

C:\Users\bahaa\anaconda3\lib\site-packages\ipykernel_launcher.py:2: OptimizeWarning: Unknown solver options: gtol
  
100%|██████████| 1/1 [00:00<00:00, 1002.46it/s]

[1]


In [77]:
np.sqrt(np.diag(result_6P_BF.hess_inv.todense())) #err

array([1., 1., 1., 1., 1., 1.])

------------------------------------------------------------------

------------------------------------------------------------------

------------------------------------------------------------------

Powell does not work with the spacial analysis; TNC is better, but still bad.

SLSQP and L-BFGS-B are good with spatial analysis (all four) but are bad with the other variables

------------------------------------------------------------------

------------------------------------------------------------------

Here is a guide to the particular scipy.optimize packages that have been tested:

< basinhopping(func, x0[, niter, T, stepsize, …])

-> Find the global minimum of a function using the basin-hopping algorithm

-> Method: L-BFGS-B -> Takes a long time to run, sometimes does not finish

-> Method: Powell -> Runs quite well at the start, but bc. it doesn't take bounds it is not accurate for some variables. Also, it doesn't give out an inverse hessian

-> Method: TNC -> Runs well, still testing



< brute(func, ranges[, args, Ns, full_output, …])

-> Minimize a function over a given range by brute force.

-> Produces incorrect results


< differential_evolution(func, bounds[, args, …])

-> Finds the global minimum of a multivariate function.

-> Continues to run endlessly


< shgo(func, bounds[, args, constraints, n, …])

-> Finds the global minimum of a function using SHG optimization.

-> Fails completely (runs very short and fails)


< dual_annealing(func, bounds[, args, …])

-> Find the global minimum of a function using Dual Annealing.

-> Fails completely

< fmin_bfgs(func, x0)

-> Does not take in bounds

-> Maybe interesting, though not likely to be promising

In [ ]:
# This basinhopping with bounds *should* work, but it is not very reliable

bnds = [(0,10), (0, 10), (0, 3.14), (-3.14, 3.14), (0, 3.14), (-3.14, 3.14)]

minimizer_kwargs = {"method": "L-BFGS-B", "bounds": bnds, "args": (accels, lin_resp, sensors_pos)}
result_6P_BH_full_bnd = scipy.optimize.basinhopping(wrapper_6P, [0, 0, 0, 0, 0, 0], minimizer_kwargs=minimizer_kwargs, niter=200)

In [80]:
# These are the truth values for the parameters above
((t_entry*s_ns)+(n_pad_strt/hnds_un))/scl_fct, ((t_exit-t_entry)*s_ns)/scl_fct, theta_entry, phi_entry, theta_exit, phi_exit

(array([0.322]),
 array([0.2052443]),
 0.4768822862810498,
 -0.8167292960052399,
 1.5463971802245688,
 -3.135141681595092)

In [ ]:
# These are the full results
result_6P_BH_full

In [40]:
# These are the parameters in the same units as the errors
((t_entry*s_ns)+(n_pad_strt/hnds_un)), ((t_exit-t_entry)*s_ns), theta_entry, phi_entry, theta_exit, phi_exit 

(array([32200.]),
 array([20524.42996829]),
 0.4768822862810498,
 -0.8167292960052399,
 1.5463971802245688,
 -3.135141681595092)

In [ ]:
hess_mtrx_inv = result_6P_BH_full['lowest_optimization_result']['hess_inv']

In [ ]:
# HERE are the Errors for the Parameters
np.sqrt(np.diag(hess_mtrx_inv) * [scl_fct**2, scl_fct**2, 1, 1, 1, 1])

In [ ]:
# This code should produce a preliminary covariance matrix plot of the parameter errors

sn.heatmap(hess_mtrx_inv, annot=True, fmt='g')

------------------------------------------------------------------

------------------------------------------------------------------

------------------------------------------------------------------